<a href="https://colab.research.google.com/github/NeethuVenugopal/Computer_Vision_Projects/blob/main/Pytorch_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import dependencies
from torch import nn, save , load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#Get Data
train = datasets.MNIST(root = "data", download = True, train = True, transform = ToTensor())
dataset = DataLoader(train,32)

class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6),10)
    )
  def forward(self,x):
    return self.model(x)

clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()

#Training flow
if __name__== "__main__":
  for epoch in range(10):
    for batch in dataset:
      x,y = batch
      x, y = x.to('cuda'), y.to('cuda')
      yhat = clf(x)
      loss = loss_fn(yhat,y)

      opt.zero_grad()
      loss.backward()
      opt.step()
    print(f"Epoch: {epoch} loss is : {loss.item()}")
  with open('model_state.pt', 'wb') as f:
    save(clf.state_dict(),f)

100%|██████████| 9912422/9912422 [00:00<00:00, 161308726.80it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 116476628.68it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 49808005.68it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 5425955.22it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Epoch: 0 loss is : 0.027100007981061935
Epoch: 1 loss is : 0.00915567111223936
Epoch: 2 loss is : 0.0008201193995773792
Epoch: 3 loss is : 0.00021194205328356475
Epoch: 4 loss is : 0.0001610060571692884
Epoch: 5 loss is : 4.059211278217845e-05
Epoch: 6 loss is : 8.94064498879743e-07
Epoch: 7 loss is : 2.235170910580564e-07
Epoch: 8 loss is : 4.507592734626087e-07
Epoch: 9 loss is : 3.199935463271686e-06


In [4]:
import torch
from PIL import Image
with open('model_state.pt', 'rb') as f:
  clf.load_state_dict(load(f))

img = Image.open('example3.png')
img = img.convert("L")
img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')
print(torch.argmax(clf(img_tensor)))


tensor(3, device='cuda:0')
